In [16]:
from google import genai

# authenticate using the API key stored in the GEMINI_API_KEY environment variable
client = genai.Client()
query1 = "Explain how modern AI language models work in 250 words."
response = client.models.generate_content(
    model = "gemini-2.5-flash", contents = query1
)

resp1 = response.text
print(response.text)

Modern AI language models, like GPT and LLaMA, operate by identifying intricate statistical patterns within vast quantities of human-generated text and code. Their foundation is typically the "Transformer" neural network architecture, which efficiently processes sequences by allowing the model to focus on different parts of the input simultaneously.

During an extensive training phase, these models engage in "self-supervised learning." A common task is predicting the next word in a sentence, or filling in a masked word, given the surrounding context. By processing trillions of words from the internet, they learn billions of parameters – numerical values that encode complex linguistic relationships, grammar, semantics, and stylistic nuances. The "attention mechanism" is key here, enabling the model to dynamically weigh the relevance of each word in the input context when making a prediction, thus understanding long-range dependencies.

When prompted, the model generates text probabilist

In [17]:
actual_len = len(resp1.split())
print(f"Response length: {actual_len} words")

Response length: 202 words


In [18]:
def create_prompt(prev_prompt:str, prev_response:str) -> str:
    prompt = """
    My previous prompt was:
    {prev_prompt}
    Your previous response was:
    {prev_response} 
    I want you to identify the criteria explicitly, reflect whether the response met it
    and the revise your response accordingly, improving it where necessary.
    """
    return prompt.format(prev_prompt=prev_prompt, prev_response=prev_response)

query2 = create_prompt(query1, resp1)

In [19]:
response2 = client.models.generate_content(
    model = "gemini-2.5-flash", contents = query2
)

print(response2.text)
print(f"Revised response length: {len(response2.text.split())} words")

The explicit criteria from your previous prompt were:
1.  **Explain how modern AI language models work.**
2.  **Limit the explanation to 250 words.**

**Reflection on Previous Response:**
The previous response successfully explained how modern AI language models work, covering key concepts like statistical patterns, Transformer architecture, self-supervised learning, attention mechanisms, and probabilistic text generation. It also provided a good summary of their capabilities and limitations.

Regarding the word count, the previous response was **240 words**, which falls within the 250-word limit. Therefore, the response met both criteria.

**Revised Response:**

Modern AI language models (e.g., GPT, LLaMA) identify complex statistical patterns within vast text and code data. Their foundation is the "Transformer" neural network architecture, efficiently processing sequences by allowing the model to focus on different input parts simultaneously.

During extensive training, models engage

## Multimodal input testing

Image input, task: text generation
[Documentation](https://ai.google.dev/gemini-api/docs/text-generation#python)

In [1]:
from PIL import Image
from google import genai

In [2]:
client = genai.Client()

image = Image.open("two_fridas_kahlo.png")

query = "Describe what you see in this image, which is a famous painting. Then, assume you are an axpert art critic, share " \
"your opinion on the painting within the delimiters <opinion> ... </opinion>"
response = client.models.generate_content(
    model = "gemini-2.5-flash", 
    contents=[image, query]
)

In [3]:
print(response.text)

This painting, titled "The Two Fridas" (Las Dos Fridas), is a renowned 1939 oil on canvas work by the Mexican artist Frida Kahlo. It depicts two seated figures, both self-portraits of Frida Kahlo, holding hands against a tumultuous, cloudy sky.

On the left, one Frida is dressed in a white, Victorian-style lace dress, which appears somewhat formal and European. Her heart is exposed and visibly bleeding, with an artery or vein extending from it, over her arm, to her hand. In this hand, she holds a pair of surgical scissors, which she is using to cut the connecting vein. Blood drips from the severed end of the vein onto her white dress, staining it. Below the cut, a small, delicate, bird-like cage hangs, appearing to catch some of the blood. Her expression is solemn, almost pained.

On the right, the second Frida wears a traditional Tehuana dress, a colorful and richly patterned attire typical of the Isthmus of Tehuantepec in Mexico, consisting of a blue blouse with gold trim and a long 

Text streaming with gemini api:

In [4]:
image2 = Image.open("kilmt.png")

response = client.models.generate_content_stream(
    model = "gemini-2.5-flash",
    contents=[image2, query]
)

for chunk in response:
    print(chunk.text, end = "")

This image depicts a full-length portrait of a woman, rendered in a style highly reminiscent of Gustav Klimt and the Viennese Secession movement, characterized by its ornate patterns, symbolic motifs, and a blend of realism and decorative abstraction.

The central figure is a woman with pale skin, dark hair pulled back, and striking dark eyes that meet the viewer's gaze directly. Her lips are a vibrant red, and her expression is serene yet enigmatic. She wears a light-colored, perhaps silvery-grey or light blue, flowing gown or inner dress. Over this, she is draped in a magnificent, voluminous shawl or cloak, also in similar cool tones, intricately decorated with stylized circular or oval motifs, resembling peacock eyes or floral patterns, in white and lighter grey. Her hands are gently clasped at the front, holding the fabric of her elaborate attire. The lower part of her garment expands outwards, revealing a richly patterned border featuring waves, swirling patterns, and red, fish-li

Multi turn conversations:

In [5]:
from google import genai

client = genai.Client()
chat = client.chats.create(model="gemini-2.5-flash")

response = chat.send_message_stream("I have 2 dogs in my house.")
for chunk in response:
    print(chunk.text, end="")

response = chat.send_message_stream("How many paws are in my house?")
for chunk in response:
    print(chunk.text, end="")

for message in chat.get_history():
    print(f'role - {message.role}', end=": ")
    print(message.parts[0].text)

That's wonderful! Dogs bring so much joy to a home.

What kind of dogs are they? Do they have names?Each dog typically has 4 paws.

So, with 2 dogs, you have **8 paws** in your house!role - user: I have 2 dogs in my house.
role - model: That's wonderful! Dogs bring so much joy to a home.

What kind of dogs are they? Do they have names?
role - user: How many paws are in my house?
role - model: Each dog typically has 4 paws.

So, with 2 dogs, you have **8 paws** in your house!
